In this example kernel, we try to demonstrate the LP for the Sudoku game. To study the problem 

$$\min_{X} \|X\|_{L^1} $$
subject to equality constraint $AX = B$.

In [131]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization
import time
from collections import defaultdict
import math

print(os.listdir("../input"))

['small2.csv', 'large2.csv', 'large1.csv', 'small1.csv']


In [132]:
small1 = pd.read_csv('../input/small1.csv')
small2 = pd.read_csv('../input/small2.csv')
large1 = pd.read_csv('../input/large1.csv')
large2 = pd.read_csv('../input/large2.csv')

In [133]:
def selected_new_quiz(result, clue, head=True):
    new_clue = clue.copy()
    coord = np.nonzero(result != new_clue)
    if head:
        select_clue = result[coord[0][0],coord[1][0]]
        new_clue[coord[0][0],coord[1][0]] = select_clue
    else:
        select_clue = result[coord[0][-1],coord[1][-1]]
        new_clue[coord[0][-1],coord[1][-1]] = select_clue 
    return new_clue

In [134]:
def clue_recovery(result, clue):
    mask = result != clue
    return mask * clue + result

In [135]:
def str2mat(string, N=9):
    N = 9
    mat = np.reshape([int(c) for c in string], (N,N))
    return mat

In [136]:
def mat2str(mat):
    string = ''
    for element in mat.reshape(-1):
        string += str(element)
    return string

In [137]:
def dup_val(arr):
    u, c = np.unique(arr, return_counts=True)
    return u[c>1]

In [138]:
def del_dup(X):
    '''
    set all contracdicted values to 0
    '''
    mat = X.copy()
    location = set()
    M = 3
    for i in range(mat.shape[0]):

        dups = dup_val(mat[i,:])
        if dups.size>0:
            for dup in dups:
                for coord in np.nonzero((mat[i,:]==dup))[0]:
                    location.add((i,coord))

        dups = dup_val(mat[:,i])
        if dups.size>0:
            for dup in dups:
                for coord in np.nonzero((mat[:,i]==dup))[0]:
                    location.add((coord,i))
                    
    for i in range(M):
        for j in range(M):
            dups = dup_val(mat[i*M:i*M+M,j*M:j*M+M])
            if dups.size>0:
                for dup in dups:
                    coord = np.nonzero(mat[i*M:i*M+M,j*M:j*M+M]==dup)
                    for x, y in zip(coord[0],coord[1]):
                        location.add((x+i*M,y+j*M))
    for loc in location:
        mat[loc]=0
    return mat

In [139]:

def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))




# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(m, N=9):
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE

In [140]:
def solve(quiz,string=False):
    
    A0 = fixed_constraints()
    if string:
        A1 = clue_constraint(str2mat(quiz))
    else :
        A1 = clue_constraint(quiz)
    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    result = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) )
    return result

In [141]:
def weighted_solve(quiz, eps=10, string=False):
    L = 10
    tol = 1e-10
    
    A0 = fixed_constraints()
    if string:
        A1 = clue_constraint(str2mat(quiz))
    else :
        A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])

    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B
    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x0 = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]
    for i in range(L):
        W = (1/(np.abs(x0)+eps))
        c = np.concatenate((W,W),axis=0)
        ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
        x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]
        if np.linalg.norm(x0-x) < tol:
            break
        x0 = x
    z = np.reshape(x, (81, 9))
    result = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) )
    return result

In [170]:
def my_block(mygb, N=9):
    dd = defaultdict(list)
    for i in range(N):
        for j in range(N):
            key = (math.floor(i/3), math.floor(j/3))
            dd[key].append(mygb[i][j])
    return dd

def forward_checking(input_quiz , N=9):
#     print(input_quiz)
    m = np.reshape([int(c) for c in input_quiz],(N,N))
    
    for i in range(N):
        for j in range(N):
           # print(i, j,m[i][j])
            if m[i][j] == 0:
                m[i][j] = 123456789
               
                for k in m[i]:
                   # print(m)
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string:
                            if len(string) > 1:
                                string = string.replace(str(k), '')
                       # print("yes",string)
                    
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("row,",m[i][j])
                        
                        
                for k in m[:,j]:
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string and len(string) > 1:
                            string = string.replace(str(k), '')
                       # print("no",string)
                       
        
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("col",m[i][j])
                        
                        
                dd = my_block(m) #block dict
#                 print("dd")
#                 print(dd)
#                 print("i,j",i,j)
#                 print("dd",dd)

                
                ll = dd[(math.floor(i/3), math.floor(j/3))]
#                 print("ll")
#                 print(ll)
#                 print("ll")
#                 print(ll)
    
                for k in ll:         #eliminate block
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string and len(string) > 1:
                            string = string.replace(str(k), '')
#                         print("hello",string)
                        
                        m[i][j] = int(string)
#                         if i == 5 and j == 6:
#                             print("box",m[i][j])
    for i in range(N):
        for j in range(N):
            if m[i][j] > 9:
                m[i][j] = 0
                
    return m


# quiz2 = "060720908084003001700100065900008000071060000002010034000200706030049800215000090"
# quiz1 = "000009007060000800789062350430600590090508020018004073043210786005000040100400000"
# a = forward_checking(quiz2,N=9)
# print("yay")
# print(a)

In [171]:
# def forward_checking(input_quiz , N=9):
#     print(input_quiz)
#     m = np.reshape([int(c) for c in input_quiz],(N,N))
    
#     for i in range(N):
#         for j in range(N):
#             print(i, j,m[i][j])
#             if m[i][j] == 0:
#                 m[i][j] = 123456789
               
#                 for k in m[i]:
#                     print("k",k)
#                     if k < 10 and k != 0:
#                         string = str(m[i][j])
#                         if str(k) in string:
#                             string = string.replace(str(k), '')
#                         print("yes",string)
#                         m[i][j] = int(string)
                        
                        
#                 for k in m[:,j]:
#                     print("k",k)
#                     if k < 10 and k != 0:
#                         string = str(m[i][j])
#                         if str(k) in string:
#                             string = string.replace(str(k), '')
#                         print("no",string)
#                         m[i][j] = int(string)
                        
# #                 dd = my_block(m) #block dict
# #                 print(dd)
                
# #                 ll = dd[(math.ceil(i/3), math.ceil(j/3))]
# #                 print("ll")
# #                 print(ll)
# #                 for k in ll:         #eliminate block
# #                     if k < 10:
# #                         string = str(m[i][j])
# #                         s = string.replace(str(k), '')
# #                         print("hello",s)
# #                         m[i][j] = int(s)
    
#     for i in range(N):
#         for j in range(N):
#             if m[i][j] > 9:
#                 m[i][j] = 0
                
#     return m

# def my_block(mygb, N=9):
#     dd = defaultdict(list)
#     for i in range(N):
#         for j in range(N):
#             print(i,j)
#             key = (math.ceil(i/3), math.ceil(j/3))
#             dd[key].append(mygb[i][j])
#     return dd

In [172]:
def clean(x):
    if x > 9:
        x = 0
    return x

In [175]:
def sudoku_solver(quiz, eps=10):
    #clue = str2mat(quiz)
    quiz = forward_checking(str(quiz))
    #quiz = np.vectorize(clean)(quiz)
    clue = quiz
    result = weighted_solve(quiz,eps)

    result_del = del_dup(result)
    if np.any(result_del==0):
#         print('sample:',i,', try step 1')
        quiz_2 = clue_recovery(result_del,clue)
        result = weighted_solve(quiz_2,eps)
        result_del = del_dup(result)
        if np.any(result_del==0):
#             print('sample:',i,', try step 2')
            quiz_2 = clue_recovery(result_del,clue)
            result = weighted_solve(quiz_2,eps)
            result_del = del_dup(result)
            if np.any(result_del==0):
#                 print('sample:',i,', try step 3 head')
                quiz_3 = selected_new_quiz(result_del, clue)
                result = weighted_solve(quiz_2,eps)
                result_del = del_dup(result)
                if np.any(result_del==0):
#                     print('sample:',i,', try step 3 tail')
                    quiz_3 = selected_new_quiz(result_del, clue, head=False)
                    result = weighted_solve(quiz_2,eps)
    return result

### small1

In [160]:
corr_cnt = 0
start = time.time()
data = small1


for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

KeyboardInterrupt: 

### small2

In [ ]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   2.28 secs. Success rate: 19 / 20 
Aver Time:   2.70 secs. Success rate: 36 / 40 
Aver Time:   2.94 secs. Success rate: 52 / 60 
Aver Time:   2.84 secs. Success rate: 70 / 80 
Aver Time:   2.93 secs. Success rate: 87 / 100 
Aver Time:   2.89 secs. Success rate: 105 / 120 
Aver Time:   3.32 secs. Success rate: 116 / 140 


### other $\epsilon$ test, defalt $=10$

In [ ]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz,eps=20)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

### $\epsilon=30,40$ SVD did not converge 

In [ ]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz,eps=30)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

### large1

In [ ]:
corr_cnt = 0
start = time.time()
data = large1
random_seed = 42
np.random.seed(random_seed)
samples = np.random.choice(len(data), 1000)
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

### large2

In [ ]:
corr_cnt = 0
start = time.time()
data = large2
random_seed = 42
np.random.seed(random_seed)
samples = np.random.choice(len(data), 1000)
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )